In [1]:
import pandas as pd
import numpy as np

# Define the sample dataset
data = {
    'CustomerID': [1, 2, 3, 4, 5],
    'LoanAmount': [10000, 15000, 20000, 12000, 8000],
    'Acceptance': ['Accept', 'Decline', 'Accept', 'Decline', 'Accept']
}

df = pd.DataFrame(data)

# Define the action space (offers)
offers = [
    {
        'Name': 'Offer A',
        'Tenure': 12,
        'InterestRate': 5.5,
        'ProcessingFee': 100,
        'InsuranceFee': 50
    },
    {
        'Name': 'Offer B',
        'Tenure': 24,
        'InterestRate': 6.2,
        'ProcessingFee': 150,
        'InsuranceFee': 75
    },
    {
        'Name': 'Offer C',
        'Tenure': 18,
        'InterestRate': 4.8,
        'ProcessingFee': 120,
        'InsuranceFee': 60
    },
    {
        'Name': 'Default Offer',
        'Tenure': 36,
        'InterestRate': 7.0,
        'ProcessingFee': 200,
        'InsuranceFee': 100
    }
]

# Define the threshold for the number of offers
threshold_offers = 3

# Initialize the Q-table
num_states = len(df)  # Number of states
num_actions = len(offers)  # Number of actions
Q = np.zeros((num_states, num_actions))

# Define the hyperparameters
learning_rate = 0.1
discount_factor = 0.9
epsilon = 0.1
num_episodes = 1000

# Function to get the recommended offer for a given state
def get_recommendation(state):
    action = np.argmax(Q[state, :])
    return offers[action]['Name']

# Function to get the next offer based on the current offer and customer's response
def get_next_offer(current_offer, response):
    current_offer_index = [offer['Name'] for offer in offers].index(current_offer)
    
    # If the current offer is the last offer or the threshold is reached, return the default offer
    if current_offer_index >= len(offers) - 1 or current_offer_index >= threshold_offers - 1:
        return 'Default Offer'
    
    # If the response is "Decline", return the next offer
    if response.lower() == 'decline':
        return offers[current_offer_index + 1]['Name']
    
    # If the response is "Accept", return the current offer
    return current_offer

# Function to offer recommendations to the customer using Q-learning
def offer_recommendation(customer_id):
    customer_data = df.loc[df['CustomerID'] == customer_id].iloc[0]
    state = df[df['CustomerID'] == customer_id].index[0]
    num_offers = 0
    current_offer = None
    
    while num_offers < threshold_offers:
        if current_offer is None:
            recommended_offer = get_recommendation(state)
        else:
            recommended_offer = get_next_offer(current_offer, response)
        print(f"Offer: {recommended_offer}")
        print_offer_details(recommended_offer)
        
        response = input("Enter customer response (Accept/Decline): ")
        
        if response.lower() == 'accept':
            print("Offer accepted!")
            break
        else:
            print("Offer declined. Offering the next set of offers.")
            num_offers += 1
            next_offer = get_next_offer(recommended_offer, response)
            next_state = state + 1 if state + 1 < len(df) else state
            reward = -1
            for action in range(num_actions):
                Q[state, action] += learning_rate * (reward + discount_factor * np.max(Q[next_state, :]) - Q[state, action])
            state = next_state
            current_offer = recommended_offer
    
    if num_offers >= threshold_offers or recommended_offer is None:
        recommended_offer = 'Default Offer'
        print(f"Default Offer")
        print_offer_details(recommended_offer)
        print("Default offer presented.")
    
    return recommended_offer


# Function to print offer details
def print_offer_details(offer):
    if offer == 'Default Offer':
        offer_details = next((o for o in offers if o['Name'] == 'Default Offer'), None)
        print(f"Default Offer Details:")
        print(f" - Tenure: {offer_details['Tenure']}")
        print(f" - Interest Rate: {offer_details['InterestRate']}")
        print(f" - Processing Fee: {offer_details['ProcessingFee']}")
        print(f" - Insurance Fee: {offer_details['InsuranceFee']}")
    else:
        offer_details = next((o for o in offers if o['Name'] == offer), None)
        print(f"Offer: {offer}")
        print(f" - Tenure: {offer_details['Tenure']}")
        print(f" - Interest Rate: {offer_details['InterestRate']}")
        print(f" - Processing Fee: {offer_details['ProcessingFee']}")
        print(f" - Insurance Fee: {offer_details['InsuranceFee']}")
        print()


# Example usage
customer_id = 3
recommended_offer = offer_recommendation(customer_id)
print(f"Recommended Offer for Customer {customer_id}: {recommended_offer}")


Offer: Offer A
Offer: Offer A
 - Tenure: 12
 - Interest Rate: 5.5
 - Processing Fee: 100
 - Insurance Fee: 50

Enter customer response (Accept/Decline): Accept
Offer accepted!
Recommended Offer for Customer 3: Offer A
